<a href="https://colab.research.google.com/github/SujanKarna/RAG_ChatModel/blob/master/Langchain_chatModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install chromadb sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import openai
import os

In [2]:
# Initiaize the ChromaDb Client
client = chromadb.Client(Settings(persist_directory="./rag_store"))

In [3]:
# Create a collection
collection = client.get_or_create_collection(name="rag_docs")

In [4]:
# Prepare Documents
documents = [
    "Retrieval-Augmented Generation (RAG) combines search and generation.",
    "FAISS is a fast similarity search library developed by Facebook.",
    "ChromaDB supports metadata filtering and persistent storage.",
    "SentenceTransformers can convert text into semantic embeddings."
]

In [5]:
# Embedd Documents
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents).tolist()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Add Documents to ChromaDB
collection.add(
    documents=documents,
    embeddings=doc_embeddings,
    ids=[f"doc_{i}" for i in range(len(documents))]
)

In [8]:
# Retrieve top-K relevant documents
def retrieve(query, k=3):
    query_embedding = embedder.encode([query]).tolist()[0]
    results = collection.query(query_embeddings = [query_embedding], n_results = k)
    return results['documents'][0]

### For Open-Source LLMs - Mistral

In [19]:
# using local llm from huggingface
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from huggingface_hub import login

In [22]:
# Generate answer with llm model
# Load Mistral model for generation

login(token= userdata.get("HUGGINGFACE_TOKEN"))
gen_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_name, torch_dtype=torch.float16, device_map="auto")




# Format prompt
def generate_answer_local(context, query):
    prompt = f"""### Instruction:
              Use the following context to answer the question.
              ### Context:{context}
              ### Question: {query}
              ### Answer:"""
    inputs = gen_tokenizer(prompt, return_tensors="pt").to(gen_model.device)
    with torch.no_grad():
        outputs = gen_model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7, top_p=0.9)
    return gen_tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Answer:")[-1].strip()



tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
context = "\n".join(retrieve("What is RAG and how does it work?"))
answer = generate_answer_local(context, "What is RAG and how does it work?")
print("🔍 Answer:\n", answer)

### For openai API

In [9]:
from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")

In [28]:
pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.101.0
    Uninstalling openai-1.101.0:
      Successfully uninstalled openai-1.101.0


In [15]:
# ✅ Securely retrieve API key from Colab Secrets
api_key = userdata.get("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

# 🧾 Generate answer using OpenAI GPT
def generate_answer(query):
    retrieved_docs = retrieve(query)
    context = "\n".join(retrieved_docs)
    prompt = f"Use the following context to answer the question:\n{context}\n\nQuestion: {query}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']


In [16]:
# Embedd the query
query = "What is RAG and how does it work?"
answer = generate_answer(query)
print("Answer:", answer)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}